# Anwendung: Studierende auf Projekte verteilen

An der Hochschule Karlsruhe gibt es im 6. Fachsemester die Lehrveranstaltung "Domänenprojekt 1" (für Data Scientists) bzw. "Anwendungsprojekt" (für WirtschaftsinformatikerInnen). Während des Semesters bearbeiten Studierende in Teams von 4 bis 6 Personen ein umfangreicheres Projekt in Kooperation mit verschiedenen Unternehmen, die dafür in die Rolle des Kunden schlüpfen. Die Veranstaltung ist mit 10 CP dotiert und bietet den Studierenden die Möglichkeit, Erfahrung im praxisnahen Projektgeschäft zu sammeln und zwar nicht nur was die Inhalte angeht, sondern auch im Hinblick auf Projektmanagement, Umgang mit Kunden, Zielen und Deadlines.

Traditionell findet am 1. Donnerstag im Semester die Zuordnung von Studierenden zu Projekten statt. Jedes Unternehmen präsentiert sein Projekt und Studierende können anschließend ihre Präferenzen (Wahl 1, Wahl 2, Wahl 3) festlegen. Weiterhin können Sie einen Wunschpartner benennen, mit dem/der sie gerne im Projekt zusammenarbeiten möchten. Idealerweise sollte jeder Studierende seine 1. Projektwahl sowie seinen Wunschpartner bekommen, doch dies ist in der Regel unmöglich.

Nach Abgabe aller Wünsche haben die betreuenden Professoren etwa 1 Stunde Zeit, um alle Wünsche unter einen Hut zu bekommen. Dies geschah bisher durch routiniertes Copy&Paste, fehleranfällige Excel-Magie und Einbringen persönlicher Vorlieben. Um diese unhaltbaren Zustände zu beseitigen, soll ein lineares Programm entwickelt werden, das die DozentInnen bei der Zuordnung unterstützt.

Dazu schauen wir uns zunächst die Eingabedaten an:

In [141]:
import pandas as pd
df = pd.read_excel("Teilnehmer_AWP.xlsx")
df

,Name,Wahl 1,Wahl 2,Wahl 3,Partner
0,Nils Dietrich,NaN,NaN,NaN,NaN
1,Julius Becker,TechWorks GmbH,MegaCorp GmbH,CorporateSolutions GmbH & Co. KG,NaN
2,Daniel Vogt,InnoTech Solutions AG,UniConsulting AG,DataVision GmbH,Kevin Schmid
3,Paul Richter,TechWorks GmbH,MegaCorp GmbH,CorporateSolutions GmbH & Co. KG,NaN
4,Tran Linh,NaN,NaN,NaN,NaN
...,...,...,...,...,...
61,Annika Schulz,UniConsulting AG,DataVision GmbH,CityMobil AG,Anna Schneider
62,Lisa Schulze,CityMobil AG,Stadtverwaltung Hintertupfingen,CorporateSolutions GmbH & Co. KG,NaN
63,Clara Sommer,InfraTech GmbH,CityTech Services GmbH,Stadtverwaltung Hintertupfingen,Dilara Yıldız
64,Kevin Schmid,InnoTech Solutions AG,UniConsulting AG,DataVision GmbH,Lena Schäfer


## Indexmengen

Wir haben insgesamt 66 Studierende aus den Studiengängen WI, IIB und DSCB. Jede/r hat bis zu 3 Projekte gewählt, wobei Wahl 1 das bevorzugte Projekt ist. Weiterhin kann ein Wunschpartner angegeben werden. Sowohl Projektvorlieben als auch Wunschpartner sind optional. Wenn sie nicht gesetzt sind, wird dies durch `NaN` gekennzeichnet.

Als nächstes definieren wir geeignete Indexmengen. Diese sind:

- $\text{Studierende}$: Menge aller Studierenden (Vor- und Nachname)
- $\text{Projekte}$: Menge aller Projekte
- $\text{Partner} \subseteq \text{Studierende}$: Menge der Studierenden, die von mindestens einem anderen als Wunschpartner genannt wurden
- $\text{Ohne\_Partner} \subseteq \text{Studierende}$: Menge der Studierenden, die keinen Wunschpartner (`NaN`) angegeben haben.

In [142]:
def get_index_sets(df):
    # All students (set as DataFrame index)
    df.index = df["Name"]
    studis = df.index.to_list()

    # All projects
    projekte = pd.concat( [df["Wahl 1"], df["Wahl 2"], df["Wahl 3"]] )
    projekte = projekte[projekte.isna() == False].unique().tolist()
    
    # Get all students mentioned as partners
    studis_partner = df[df["Partner"].isin(studis)].index.tolist()

    # Find partners that are not in the list of students
    invalid_partners = df[~df["Partner"].isin(studis) & (~df["Partner"].isna())]["Partner"]
    
    # Students that do not name a partner
    studis_no_partner = df[df["Partner"].isna()].index.tolist()
    
    return studis, projekte, studis_partner, invalid_partners, studis_no_partner

studis, projekte, studis_partner, invalid_partners, studis_no_partner = get_index_sets(df)
df = df.drop("Name", axis=1)

sep = "\n" + 25*"-" + "\n"
print(f"Liste der Studierenden: ")
print(studis)
print(sep)
print(f"Anzahl Studierende: {len(studis)}")

print(sep)
print(f"Projekte, die gewählt wurden (insgesamt {len(projekte)}): ")
print(*projekte, sep="\n")

print(sep)
print(f"Personen, die als Partner gewählt wurden: ")
print(studis_partner)

print(sep)
print("Personen, die keinen Partner gewählt haben: ")
print(*studis_no_partner, sep="\n")

Liste der Studierenden: 
['Nils Dietrich', 'Julius Becker', 'Daniel Vogt', 'Paul Richter', 'Tran Linh', 'Tom Schuster', 'Jana Zimmermann', 'Jamal Al-Farsi', 'Cem Çetin', 'Lea Fischer', 'Isabella Braun', 'Dilara Yıldız', 'Jonah Wagner', 'Pham Thi', 'Ali Toprak', 'Murat Aktaş', 'Mustafa Öztürk', 'Julian Klein', 'Laura Berger', 'Mia Schulz', 'Maximilian Schuster', 'Lea Huber', 'Robin Scholz', 'Maria Baumann', 'Emilia Wolf', 'Niklas Richter', 'Victoria Zimmer', 'Anna Schneider', 'Carolin Wolf', 'Omar Abdelrahman', 'Emma Wagner', 'Nada Mohamed', 'Pauline Neumann', 'Maximilian Meyer', 'Sophie Becker', 'Leah Becker', 'Omar Al-Saleh', 'Selin Korkmaz', 'Amelie Friedrich', 'Sophia Hoffmann', 'Nguyen Thanh', 'David Hartmann', 'Benjamin Braun', 'Moritz Richter', 'Johannes Keller', 'David Weber', 'Sara Lehmann', 'Youssef Ibrahim', 'Finn Weber', 'Ahmet Yıldırım', 'Marlene Schmidt', 'Le Anh', 'Khaled Mansour', 'Leon Fischer', 'Sebastian Berger', 'Hannah Bauer', 'Fabian Klein', 'Mohammed Hassan', 'Luc

## Pre-processing: Punkte

Nun kommt der Modellierungsteil: Dass jede/r seinen/ihren Wunschpartner und Wunschprojekt bekommt, ist in der Regel nicht möglich. Es muss also ein Kompromiss gefunden werden. Wir bilden dies über Punkte ab: Jede Zuordnung von Projekten ergibt bei jedem Studierenden eine Anzahl von Punkten, je nachdem, ob er die Erst-/Zweit-/Drittwahl (oder gar keine) und seinen Wunschpartner bekommen hat.

Konkret legen wir folgende Punktzahlen fest:
- Erstwahl erhalten: $w_1=3$ Punkte
- Zweitwahl erhalten: $w_2=2$ Punkte
- Drittwahl erhalten: $w_3=1$ Punkt
- Keines der drei angegebenen Projekte erhalten: $w_4=-2$ Punkte
- Wunschpartner bekommen: $w=3$ Punkte
- Hat ein Studierender `NaN` angegeben, so bekommt er/sie bei jedem zugewiesenen Projekt die entsprechenden Punkte (Beispiel: Ein Studierender hat nur eine Erstwahl angegeben aber lässt Zweit- und Drittwahl frei. Wenn er seine Erstwahl bekommt, so erhält er drei Punkte, für jedes andere Projekt erhält er zwei Punkte)

Wir ergänzen den DataFrame mit den Punkten für die Projektwahl. Jedes Projekt ist eine Spalte, in der Spalte stehen die Punkte, die man bei der Zuordnung zu diesem Projekt erhalten würden.

In [143]:
# Punkte (Parameter)
w1_pkt = 3
w2_pkt = 2
w3_pkt = 1
keine_wahl_pkt = -2
partner_pkt = 3

# Wer bei Wahl i kein Projekt angegeben hat, bekommt bei allen Projekten die Punktzahl für Wahl i
for prj in projekte: 
    df[prj] = w1_pkt*( (df["Wahl 1"] == prj)                                                  |  df["Wahl 1"].isna()) + \
              w2_pkt*(((df["Wahl 2"] == prj) & (df["Wahl 1"] != prj))                         | (df["Wahl 2"].isna() & ~df["Wahl 1"].isna())) + \
              w3_pkt*(((df["Wahl 3"] == prj) & (df["Wahl 1"] != prj) & (df["Wahl 2"] != prj)) | (df["Wahl 3"].isna() & ~df["Wahl 2"].isna()))
    
    df.loc[df[prj] == 0, prj] = keine_wahl_pkt

df.head()

,Wahl 1,Wahl 2,Wahl 3,Partner,TechWorks GmbH,InnoTech Solutions AG,DataVision GmbH,CorporateSolutions GmbH & Co. KG,MegaCorp GmbH,InfraTech GmbH,SmartTech Solutions GmbH,CityTech Services GmbH,MediCare Hospital,UniConsulting AG,Stadtverwaltung Hintertupfingen,HealthCare GmbH,CityMobil AG,Digital Services GmbH,AdvancedTech Solutions GmbH
Name,,,,,,,,,,,,,,,,,,,
Nils Dietrich,NaN,NaN,NaN,NaN,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Julius Becker,TechWorks GmbH,MegaCorp GmbH,CorporateSolutions GmbH & Co. KG,NaN,3,-2,-2,1,2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
Daniel Vogt,InnoTech Solutions AG,UniConsulting AG,DataVision GmbH,Kevin Schmid,-2,3,1,-2,-2,-2,-2,-2,-2,2,-2,-2,-2,-2,-2
Paul Richter,TechWorks GmbH,MegaCorp GmbH,CorporateSolutions GmbH & Co. KG,NaN,3,-2,-2,1,2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
Tran Linh,NaN,NaN,NaN,NaN,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


## Modellentwicklung

### Problemdaten
Wir bezeichnen mit $s_{i,j}$ die Punkte, die Studierender $i$ bei der Zuordnung zu Projekt $j$ erhalten würden.

### Variablen

\begin{align*}
x_{ij} &= \left\{\begin{array}{rl}1, & \text{Studi }i\text{ bekommt Projekt }j\\ 
                            0, & \text{sonst.}\end{array}\right.,\quad i\in \text{Studierende}, j\in \text{Projekte}\\
z_{j} &= \left\{\begin{array}{rl}1, & \text{Projekt }j\text{ findet statt.}\\ 
                            0, & \text{sonst.}\end{array}\right.,\quad j\in \text{Projekte}\\
p_{ij} &= \left\{\begin{array}{rl}1, & \text{Studi }i\text{ ist in Projekt }j\text{ mit dem Wunschpartner zusammen}\\ 
                            0, & \text{sonst.}\end{array}\right.,\quad i\in \text{Studierende}, j\in \text{Projekte}\\
m&\in\mathbb{R}: \text{ minimale Anzahl Punkte (Hilfsvariable für maxmin-Formulierung)}
\end{align*}

### Zielfunktion

Es soll die Punktzahl des Studierenden mit den wenigsten Punkten maximiert werden ("dem ärmsten Wicht soll es möglichst gut gehen").

$$
\max_{x_{ij}, z_j, p_{ij}, m} m
$$

Es ist allerdings davon auszugehen, dass es mehrere Lösungen mit dem gleichen Zielfunktionswert gibt. Wenn es zwei Lösungen gibt, bei denen die minimale Punktzahl gleich ist, soll diejenige bevorzugt werden, bei denen die Summe der Punkte über alle Teilnehmer höher ist ("maximiere Gesamtwohl"). Dies können wir erreichen indem wir die Gesamtpunktzahl zur Zielfunktion dazuaddieren, aber mit einem Vorfaktor $\epsilon$ versehen, der sicherstellt, dass der erste Summand (kleinste Punktzahl über alle Teilnehmer) immer die Summe dominiert, d.h. eine schlechte minimale Punktzahl kann nie durch eine gute Gesamtpunktzahl ausgeglichen werden. $\epsilon$ muss also nur klein genug sein, ansonsten ist die Wahl unkritisch. Wir setzen $\epsilon=0.0001$. Insgesamt erhalten wir als Zielfunktion:

$$
\max_{x_{ij}, z_j, p_{ij}, m} m + \epsilon \sum_{i\in \text{Studierende}, j\in \text{Projekte}} x_{ij}s_{ij}+p_{ij}w
$$

Wir implementieren Variablen und Zielfunktion schon einmal in einem Gurobi Modell.

In [144]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("AWPZuordnungSoSe2024")

#
# VARIABLES
#

# Hilfsvariable für max-min Zielfunktion
min_pkte = m.addVar()

# Bekommt Studi i Projekt j? 
x = m.addVars(studis, projekte, vtype=GRB.BINARY)

# Findet Projekt j statt?
z = m.addVars(projekte, vtype=GRB.BINARY)

# Ist Studi i in Projekt j mit seinem Wunschpartner zusammen?
p = m.addVars(studis, projekte, vtype=GRB.BINARY)

#
# OBJECTIVE
#

# Maximiere minimale Punkte über alle Teilnehmer (primäres Ziel). Danach Gesamtpunkte.
eps = 1e-4
m.setObjective(min_pkte + \
               eps * (gp.quicksum(x[i,j]*df.loc[i,j] for i in studis for j in projekte) + \
                      gp.quicksum(p[i,j]*partner_pkt for i in studis for j in projekte)), 
               GRB.MAXIMIZE)

### Constraints

Wir definieren folgende Constraints:

1. Jeder Studierende muss genau einem Projekt zugeordnet werden:

  $$\sum_{j\in\text{Projekte}}x_{ij}=1,\quad i\in\text{Studierende}$$

2. Wer keinen Wunschpartner angibt, bekommt keine Partnerpunkte:

  $$ p_{ij}=0, \quad i\in\text{Ohne\_Partner}, j\in\text{Projekte}$$

3. Studierender $i$ bekommt Partnerpunkte, wenn er mit dem Wunschpartner im selben Projekt $j$ ist, also wenn $x_{ij}=1$ und $x_{\text{Partner}(i),j}=1$:

  $$x_{ij}+x_{\text{Partner}(i),j}\geq 2p_{ij},\quad i\in\text{Partner}$$
  
   Wir iterieren hier über alle genannten Partner. $\text{Partner}(i)$ ist derjenige Studierende, der sich $i$ als Partner gewünscht hat.
  
4. Jedes Projekt wird mit 4 bis 6 Studierenden besetzt oder es findet nicht statt:

  $$4z_j\leq \sum_{i\in\text{Studierende}}x_{ij}\leq 6z_j,\quad j\in\text{Projekte}$$

5. Jeder Studierende muss eine Mindestpunktzahl haben (diese wird optimiert):

  $$\sum_{j\in\text{Projekte}}x_{ij}s_{ij}+p_{ij}w\geq m,\quad i\in\text{Studierende}$$

6. Studierende, die keinen Partner gewählt haben, müssen ihre Erstwahl bekommen:

  $$\sum_{j\in\text{Projekte}}x_{ij}s_{ij}\geq w_1,\quad i\in\text{Ohne\_Partner}$$

Anschließend lösen wir das Modell mit Gurobi.

In [145]:
# Jeder genau ein Projekt
m.addConstrs(gp.quicksum(x[i,j] for j in projekte) == 1 for i in studis)

# Wer keinen Wunschpartner angibt, bekommt keine Partnerpunkte
m.addConstrs(p[i,j] == 0 for i in studis_no_partner for j in projekte)

# Nur wenn x[wunschpartner von i, j] und x[i,j] beide 1 sind, kann p[i,j]=1 werden.
# Wenn jemand keinen Partner angegeben hat ist df.loc[i,"Partner"] nan, also ist x[nan,j] ungültig. 
# Die Constraint wird dann nicht hinzugefügt.
m.addConstrs(x[df.loc[i,"Partner"],j] + x[i,j] >= 2*p[i,j] for i in studis_partner for j in projekte)

# Jedes Projekt zwischen 4 und 6 Personen (oder es findet nicht statt)
m.addConstrs(gp.quicksum(x[i,j] for i in studis) >= z[j]*4 for j in projekte)
m.addConstrs(gp.quicksum(x[i,j] for i in studis) <= z[j]*6 for j in projekte)

# Jeder Studi muss Mindestpunktzahl haben
m.addConstrs(gp.quicksum(x[i,j]*df.loc[i,j] for j in projekte) + \
             gp.quicksum(p[i,j]*partner_pkt for j in projekte) >= min_pkte for i in studis )

# Studis ohne Partner müssen ihre Erstwahl bekommen
m.addConstrs(gp.quicksum(x[i,j]*df.loc[i,j] for j in projekte) >= w1_pkt for i in studis_no_partner )
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Linux Mint 21.3")

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1180 rows, 1996 columns and 7596 nonzeros
Model fingerprint: 0xcad4efd4
Variable types: 1 continuous, 1995 integer (1995 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e-04, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 937 rows and 1502 columns
Presolve time: 0.02s
Presolved: 243 rows, 494 columns, 1910 nonzeros
Variable types: 0 continuous, 494 integer (493 binary)
Found heuristic solution: objective 1.0169000
Found heuristic solution: objective 1.0205000

Root relaxation: objective 2.947392e+00, 365 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl

## Auslesen der Lösung

Wir lesen folgende Aspekte der Lösung aus:
- Welche Projekte finden statt?
- Das vorgeschlagene Projekt für jeden Studierenden
- Hat der Studierende seinen Wunschpartner bekommen?
- Hat der Studierende seine Erst-, Zweit- oder Drittwahl bekommen?

Anschließend visualisieren wir die Lösung über die Kohorte in einem Säulendiagramm mit folgender Interpretation:
- Den Anteil derjenigen, die ihre Erst-, Zweit- bzw. Drittwahl bekommen haben und diejenigen, die keine Wahl angegeben haben
- Farblich unterschieden, ob die Studierenden ihren Wunschpartner bekommen haben oder keine angegeben haben.

In [146]:
for j in projekte:
    if z[j].x == 0:
        print(f"{j:33} findet nicht statt.")
    else:
        print(f"{j:33} findet statt.")
    
k = 0
for i in studis:
    for j in projekte:
        if x[i,j].x == 1:
            #print(f"{k:2}   {i:25} {j:10} {p[i,j].x}")
            k += 1
            df.loc[i,"Projektzuordnung_Vorschlag"] = j
            
            if pd.isna(df.loc[i,"Partner"]):
                df.loc[i,"Wunschpartner"] = "n/a"
            elif p[i,j].x == 1:
                df.loc[i,"Wunschpartner"] = "ja"
            else:
                df.loc[i,"Wunschpartner"] = "nein"
            
            if j == df.loc[i,"Wahl 1"]:
                df.loc[i,"Wahl bekommen"] = "Wahl 1"
            elif j == df.loc[i,"Wahl 2"]:
                df.loc[i,"Wahl bekommen"] = "Wahl 2"
            elif j == df.loc[i,"Wahl 3"]:
                df.loc[i,"Wahl bekommen"] = "Wahl 3"
            else:
                df.loc[i,"Wahl bekommen"] = "Keine"

TechWorks GmbH                    findet statt.
InnoTech Solutions AG             findet statt.
DataVision GmbH                   findet statt.
CorporateSolutions GmbH & Co. KG  findet statt.
MegaCorp GmbH                     findet statt.
InfraTech GmbH                    findet statt.
SmartTech Solutions GmbH          findet statt.
CityTech Services GmbH            findet statt.
MediCare Hospital                 findet nicht statt.
UniConsulting AG                  findet statt.
Stadtverwaltung Hintertupfingen   findet statt.
HealthCare GmbH                   findet statt.
CityMobil AG                      findet statt.
Digital Services GmbH             findet statt.
AdvancedTech Solutions GmbH       findet nicht statt.


In [147]:
df = df.drop(columns=projekte)
df

,Wahl 1,Wahl 2,Wahl 3,Partner,Projektzuordnung_Vorschlag,Wunschpartner,Wahl bekommen
Name,,,,,,,
Nils Dietrich,NaN,NaN,NaN,NaN,MegaCorp GmbH,n/a,Keine
Julius Becker,TechWorks GmbH,MegaCorp GmbH,CorporateSolutions GmbH & Co. KG,NaN,TechWorks GmbH,n/a,Wahl 1
Daniel Vogt,InnoTech Solutions AG,UniConsulting AG,DataVision GmbH,Kevin Schmid,InnoTech Solutions AG,ja,Wahl 1
Paul Richter,TechWorks GmbH,MegaCorp GmbH,CorporateSolutions GmbH & Co. KG,NaN,TechWorks GmbH,n/a,Wahl 1
Tran Linh,NaN,NaN,NaN,NaN,Digital Services GmbH,n/a,Keine
...,...,...,...,...,...,...,...
Annika Schulz,UniConsulting AG,DataVision GmbH,CityMobil AG,Anna Schneider,UniConsulting AG,ja,Wahl 1
Lisa Schulze,CityMobil AG,Stadtverwaltung Hintertupfingen,CorporateSolutions GmbH & Co. KG,NaN,CityMobil AG,n/a,Wahl 1
Clara Sommer,InfraTech GmbH,CityTech Services GmbH,Stadtverwaltung Hintertupfingen,Dilara Yıldız,InfraTech GmbH,ja,Wahl 1


In [148]:
import plotly.io as pio
import plotly.express as px
pio.templates.default = "seaborn"

px.histogram(data_frame=df, x="Wahl bekommen", color="Wunschpartner", category_orders={"Wahl bekommen": ["Wahl 1", "Wahl 2", "Wahl 3", "Keine"],
                                                                                       "Wunschpartner": ["ja", "n/a", "nein"]}, height=600)

## Aufgaben

1. Gehen Sie die MILP-Formulierung durch und versuchen Sie, die einzelnen Constraint-Formulierungen nachzuvollziehen.
2. Modifizieren Sie das Modell. Versetzen Sie sich dazu in die Situation eines der Teilnehmer. 
    1. Wie wichtig ist es ihnen, mit ihrem Wunschpartner zusammenzuarbeiten vs. an ihrem Wunschprojekt zu arbeiten? Wie ist dies im Moment im Modell gewichtet?
    2. Sollten Teilnehmer, die keinen Partner angeben, immer ihr Wunschprojekt (Wahl 1) bekommen? 
        1. Ist dies überhaupt immer, d.h. bei allen denkbaren Kombinationen von Wunschprojekten, möglich? (Wenn ja: Begründung, wenn nein: Gegenbeispiel)
        2. Entfernen Sie die entsprechende Constraint aus dem Problem. Ändert sich die Lösung?
    3. Optimieren Sie die Gesamtpunktzahl anstelle der Punktzahl des Studierenden mit der minimalen Punktzahl. Gibt es nun einen Studierenden mit einer kleineren Punktzahl als vorher?
3. Gibt es weitere Aspekte, die sie bei der Optimierung mit einbeziehen würden, ggf. auch durch Erhebung weiterer Daten, zusätzlich zu Wahl 1, Wahl 2, Wahl 3 und Wunschpartner?